## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sola,VU,-13.88,167.55,82.40,78,75,11.41,broken clouds
1,1,Punta Arenas,CL,-53.15,-70.92,55.40,66,75,14.99,broken clouds
2,2,Bongandanga,CD,1.50,21.05,70.97,97,98,1.70,overcast clouds
3,3,Port Alfred,ZA,-33.59,26.89,66.90,78,84,22.28,broken clouds
4,4,Mahebourg,MU,-20.41,57.70,71.60,64,40,10.29,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bongandanga,CD,1.50,21.05,70.97,97,98,1.70,overcast clouds
3,3,Port Alfred,ZA,-33.59,26.89,66.90,78,84,22.28,broken clouds
4,4,Mahebourg,MU,-20.41,57.70,71.60,64,40,10.29,scattered clouds
6,6,Souillac,MU,-20.52,57.52,71.60,64,40,10.29,scattered clouds
7,7,Airai,TL,-8.93,125.41,60.48,82,97,2.08,overcast clouds
10,10,San Quintin,MX,30.48,-115.95,77.68,36,0,9.22,clear sky
11,11,Rocha,UY,-34.48,-54.33,64.89,53,0,8.97,clear sky
14,14,Sao Filipe,CV,14.90,-24.50,79.75,73,0,10.63,clear sky
15,15,Laibin,CN,23.70,109.27,69.44,70,100,10.45,overcast clouds
18,18,Ribeira Grande,PT,38.52,-28.70,64.40,88,75,14.99,light intensity drizzle rain


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()


City_ID                276
City                   276
Country                274
Lat                    276
Lng                    276
Max Temp               276
Humidity               276
Cloudiness             276
Wind Speed             276
Current Description    276
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()

narrowed_city_df.head(5) 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,San Quintin,MX,30.48,-115.95,77.68,36,0,9.22,clear sky
41,41,Tiznit,MA,29.58,-9.50,72.79,28,0,0.85,clear sky
96,96,Coihaique,CL,-45.58,-72.07,77.00,23,0,4.70,clear sky
111,111,Saldanha,ZA,-33.01,17.94,78.80,24,0,6.93,clear sky
205,205,Hervey Bay,AU,-25.30,152.85,71.01,93,0,3.00,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = narrowed_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(5)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,San Quintin,MX,77.68,clear sky,30.48,-115.95,
41,Tiznit,MA,72.79,clear sky,29.58,-9.50,
96,Coihaique,CL,77.00,clear sky,-45.58,-72.07,
111,Saldanha,ZA,78.80,clear sky,-33.01,17.94,
205,Hervey Bay,AU,71.01,clear sky,-25.30,152.85,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [9]:
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,San Quintin,MX,77.68,clear sky,30.48,-115.95,Old Mill Hotel
41,Tiznit,MA,72.79,clear sky,29.58,-9.50,
96,Coihaique,CL,77.00,clear sky,-45.58,-72.07,Diego De Almagro Coyhaique
111,Saldanha,ZA,78.80,clear sky,-33.01,17.94,Blue Bay Lodge
205,Hervey Bay,AU,71.01,clear sky,-25.30,152.85,Shelly Bay Resort
220,Zwedru,LR,74.53,light rain,6.07,-8.13,Elijah Worjloh
251,Alice Springs,AU,71.60,clear sky,-23.70,133.88,Desert Palms Alice Springs
258,Tripoli,LY,70.47,clear sky,32.88,13.19,Al Waddan Hotel
281,Azul,AR,73.00,clear sky,-36.78,-59.86,Cabañas Dulcinea
299,Anahuac,MX,79.79,clear sky,27.23,-100.15,Rancho Guadalupe


In [23]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel = hotel_df.dropna()
print(clean_hotel)


              City Country  Max Temp Current Description    Lat     Lng  \
10     San Quintin      MX     77.68           clear sky  30.48 -115.95   
96       Coihaique      CL     77.00           clear sky -45.58  -72.07   
111       Saldanha      ZA     78.80           clear sky -33.01   17.94   
205     Hervey Bay      AU     71.01           clear sky -25.30  152.85   
220         Zwedru      LR     74.53          light rain   6.07   -8.13   
251  Alice Springs      AU     71.60           clear sky -23.70  133.88   
258        Tripoli      LY     70.47           clear sky  32.88   13.19   
281           Azul      AR     73.00           clear sky -36.78  -59.86   
299        Anahuac      MX     79.79           clear sky  27.23 -100.15   
361          Aswan      EG     75.31           clear sky  24.09   32.91   
382          Bilma      NE     74.82           clear sky  18.69   12.92   
389   Cameron Park      US     79.00           clear sky  38.67 -120.99   
481       Buraydah      S

In [24]:
hotel_df = clean_hotel
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,San Quintin,MX,77.68,clear sky,30.48,-115.95,Old Mill Hotel
96,Coihaique,CL,77.00,clear sky,-45.58,-72.07,Diego De Almagro Coyhaique
111,Saldanha,ZA,78.80,clear sky,-33.01,17.94,Blue Bay Lodge
205,Hervey Bay,AU,71.01,clear sky,-25.30,152.85,Shelly Bay Resort
220,Zwedru,LR,74.53,light rain,6.07,-8.13,Elijah Worjloh
251,Alice Springs,AU,71.60,clear sky,-23.70,133.88,Desert Palms Alice Springs
258,Tripoli,LY,70.47,clear sky,32.88,13.19,Al Waddan Hotel
281,Azul,AR,73.00,clear sky,-36.78,-59.86,Cabañas Dulcinea
299,Anahuac,MX,79.79,clear sky,27.23,-100.15,Rancho Guadalupe
361,Aswan,EG,75.31,clear sky,24.09,32.91,Nuba Nile


In [25]:
# 8a. Create the output File (CSV)

output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Country}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))